# Import lib

In [1]:
import math

import pandas as pd
import numpy as np
from numba import cuda, void, float64, int32
import numba as nb
import time
import copy

# Preprocess (temp)

In [2]:
df = pd.read_csv('data/CC GENERAL.csv')

df['MINIMUM_PAYMENTS'] = df['MINIMUM_PAYMENTS'].fillna(df['MINIMUM_PAYMENTS'].median())
df['CREDIT_LIMIT'] = df['CREDIT_LIMIT'].fillna(df['CREDIT_LIMIT'].mean())

df = df.drop('CUST_ID', axis=1)
np_data = df.to_numpy()

# MSE

In [3]:
def mse(result_1, result_2):
  return (np.square(result_1 - result_2)).mean()

# Init k and centroids

In [4]:
k = 20
centroid = np_data[np.random.randint(np_data.shape[0], size=k), :]

# Distance

## CPU

In [5]:
def calc_distance_cpu(data, data_centroid):
    dist = np.zeros((data.shape[0], data_centroid.shape[0]))
    for i in range(data.shape[0]):
        for j in range(data_centroid.shape[0]):
            dist[i][j] = np.linalg.norm(data[i] - data_centroid[j])
    return dist

## GPU

### Preparation

Calculate thread per block and block per grid

In [6]:
def calc_dimension_for_distance(data, data_centroid):
    def next_power_of_2(x):
      return 1 << (x - 1).bit_length()
    real_dim_x = data_centroid.shape[0]
    dim_x = next_power_of_2(real_dim_x)
    dim_y = dim_x
    thread_per_blocks = (dim_x, dim_y)
    blocks_per_grid_x = 1
    blocks_per_grid_y = math.ceil(data.shape[0] / thread_per_blocks[0])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)
    return thread_per_blocks, blocks_per_grid

In [7]:
dist_tpb, dist_bpg = calc_dimension_for_distance(np_data, centroid)

Copy data to device

In [8]:
np_data_device = cuda.to_device(np_data)

### GPU distance

Ver 1

In [9]:
@cuda.jit(void(nb.types.Array(dtype=float64, ndim=2, layout="F"), nb.types.Array(dtype=float64, ndim=2, layout="C"), nb.types.Array(dtype=float64, ndim=2, layout="C")))
def calc_distance_kernel_ver1(data, data_centroid, result):
    r = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    c = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if r < data.shape[0] and c < data_centroid.shape[0]:
        total = 0
        for i in range(data_centroid.shape[1]):
            total += math.pow(data[r][i] - data_centroid[c][i], 2)
        result[r, c] = math.sqrt(total)

In [10]:
def calc_distance_gpu_ver1(data, data_centroid):
    result = np.zeros((data.shape[0], data_centroid.shape[0]))
    # data_device = cuda.to_device(data) # old version
    data_device = data
    centroid_device = cuda.to_device(data_centroid)
    result_device = cuda.to_device(result)

    # invoke kernel
    calc_distance_kernel_ver1[dist_bpg, dist_tpb](data_device, centroid_device, result_device)
    result = result_device.copy_to_host()
    return result

Ver 2 (use share memory)

In [11]:
@cuda.jit(void(nb.types.Array(dtype=float64, ndim=2, layout="F"), nb.types.Array(dtype=float64, ndim=2, layout="C"), nb.types.Array(dtype=float64, ndim=2, layout="C")))
def calc_distance_kernel_ver2(data, data_centroid, result):
    r = cuda.blockIdx.y * cuda.blockDim.y + cuda.threadIdx.y
    c = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
    if r < data.shape[0] and c < data_centroid.shape[0]:
        total = 0
        for i in range(data_centroid.shape[1]):
            total += math.pow(data[r][i] - data_centroid[c][i], 2)
        result[r, c] = math.sqrt(total)

In [12]:
def calc_distance_gpu_ver2(data, data_centroid):
    result = np.zeros((data.shape[0], data_centroid.shape[0]))
    # data_device = cuda.to_device(data) # old version
    data_device = data
    centroid_device = cuda.to_device(data_centroid)
    result_device = cuda.to_device(result)

    # invoke kernel
    calc_distance_kernel_ver1[dist_bpg, dist_tpb](data_device, centroid_device, result_device)
    result = result_device.copy_to_host()
    return result

## Compare GPU and CPU distance

GPU_VER1

In [13]:
dist_v1_gpu_time_start = time.perf_counter()
calculated_dist_gpu_ver1 = calc_distance_gpu_ver1(np_data_device, centroid)
dist_v1_gpu_time_end = time.perf_counter()

In [14]:
dist_v1_gpu_time_end - dist_v1_gpu_time_start

0.01580299399999774

GPU_VER2

In [15]:
dist_v2_gpu_time_start = time.perf_counter()
calculated_dist_gpu_ver2 = calc_distance_gpu_ver2(np_data_device, centroid)
dist_v2_gpu_time_end = time.perf_counter()

In [16]:
dist_v2_gpu_time_end - dist_v2_gpu_time_start

0.013992086000001791

CPU

In [17]:
dist_cpu_time_start = time.perf_counter()
calculated_dist_cpu = calc_distance_cpu(np_data, centroid)
dist_cpu_time_end = time.perf_counter()

In [18]:
dist_cpu_time_end - dist_cpu_time_start

1.1059301939999955

MSE CPU and GPU_VER1

In [19]:
mse(calculated_dist_cpu, calculated_dist_gpu_ver1)

3.9443850149494024e-25

MSE CPU and GPU_VER2

In [20]:
mse(calculated_dist_cpu, calculated_dist_gpu_ver2)

3.9443850149494024e-25

# Nearest centroid

In [21]:
calculated_dist_cpu = calc_distance_cpu(np_data, centroid)

## CPU

Ver 1 using numpy

In [22]:
def get_nearest_centroid_cpu_ver1(distance):
    return np.argmin(distance, axis=1)

Ver 2

In [23]:
def get_nearest_centroid_cpu_ver2(distance):
    num_samples, num_centroids = distance.shape
    nearest_centroid = np.zeros(num_samples, dtype=np.int32)

    for i in range(num_samples):
        min_dist = distance[i, 0]
        min_idx = 0
        for j in range(1, num_centroids):
            if distance[i, j] < min_dist:
                min_dist = distance[i, j]
                min_idx = j
        nearest_centroid[i] = min_idx

    return nearest_centroid

## GPU

### Preparation

In [24]:
def calc_dimension_for_nearest_centroid(distance):
    block_size = 64
    grid_size = (distance.shape[0] + block_size - 1) // block_size
    return block_size, grid_size

In [25]:
nearest_tpb, nearest_bpg = calc_dimension_for_nearest_centroid(calculated_dist_cpu)

### GPU nearest centroids

In [26]:
@cuda.jit(void(nb.types.Array(dtype=float64, ndim=2, layout="C"), nb.types.Array(dtype=int32, ndim=1, layout="C")))
def find_min_distance_kernel(distance, nearest_centroid):
  r = cuda.blockIdx.x * cuda.blockDim.x + cuda.threadIdx.x
  shape = distance.shape
  if r < shape[0]:
    data = distance[r]
    min_distance = data[0]
    min_idx = 0
    for c in range(1, shape[1]):
        current_value = data[c]
        if current_value < min_distance:
            min_distance = current_value
            min_idx = c
    nearest_centroid[r] = min_idx

In [27]:
def get_nearest_centroid_gpu(distance):  # Using GPU
    nearest_centroid = np.zeros(distance.shape[0], dtype=np.int32)
    distance_device = cuda.to_device(distance)
    nearest_centroid_device = cuda.to_device(nearest_centroid)
    find_min_distance_kernel[nearest_bpg, nearest_tpb](distance_device, nearest_centroid_device)

    resolved_nearest_centroid = nearest_centroid_device.copy_to_host()
    return resolved_nearest_centroid

## Comapre GPU and CPU

GPU

In [28]:
nearest_gpu_time_start = time.perf_counter()
calculated_nearest_gpu = get_nearest_centroid_gpu(calculated_dist_cpu)
nearest_gpu_time_end = time.perf_counter()

In [29]:
nearest_gpu_time_end - nearest_gpu_time_start

0.0025784999999984848

CPU

Ver 1

In [30]:
nearest_cpu_ver1_time_start = time.perf_counter()
calculated_nearest_cpu_ver1 = get_nearest_centroid_cpu_ver1(calculated_dist_cpu)
nearest_cpu_ver1_time_end = time.perf_counter()

In [31]:
nearest_cpu_ver1_time_end - nearest_cpu_ver1_time_start

0.0012939669999951775

Ver 2

In [32]:
nearest_cpu_ver2_time_start = time.perf_counter()
calculated_nearest_cpu_ver2 = get_nearest_centroid_cpu_ver2(calculated_dist_cpu)
nearest_cpu_ver2_time_end = time.perf_counter()

In [33]:
nearest_cpu_ver2_time_end - nearest_cpu_ver2_time_start

0.04428212500000939

MSE CPU Ver 1 and GPU

In [34]:
mse(calculated_nearest_cpu_ver1, calculated_nearest_gpu)

0.0

MSE CPU Ver 2 and GPU

In [35]:
mse(calculated_nearest_cpu_ver2, calculated_nearest_gpu)

0.0

# KMEAN

In [36]:
def get_new_centroids(data, data_nearest_centroid, number_of_centroid):
    result_centroids = np.zeros((number_of_centroid, data.shape[1]))
    for i in range(number_of_centroid):
        result_centroids[i] = data[np.where(data_nearest_centroid == i)].mean(axis=0)
    return result_centroids

## CPU

In [38]:
def kmean_cpu(data, initial_centroid):
    centroid_cpu = copy.deepcopy(initial_centroid)
    has_changed_centroid = True
    while has_changed_centroid:
        calculated_dist = calc_distance_cpu(data, centroid_cpu)  # calculated dist
        nearest_centroid = get_nearest_centroid_cpu_ver1(calculated_dist)  # assigned to centroid
        new_centroid = get_new_centroids(data, nearest_centroid, k)
        if np.all(new_centroid == centroid_cpu):
            has_changed_centroid = False
        else:
            centroid_cpu = new_centroid
    return centroid_cpu

## GPU

In [39]:
def kmean_gpu(data, initial_centroid):
    centroid_gpu = copy.deepcopy(initial_centroid)
    has_changed_centroid = True
    while has_changed_centroid:
        calculated_dist = calc_distance_gpu_ver1(np_data_device, centroid_gpu)  # calculated dist
        nearest_centroid = get_nearest_centroid_gpu(calculated_dist)  # assigned to centroid
        new_centroid = get_new_centroids(data, nearest_centroid, k)
        if np.all(new_centroid == centroid_gpu):
            has_changed_centroid = False
        else:
            centroid_gpu = new_centroid
    return centroid_gpu

## Compare CPU vs GPU

### RUN CPU

In [40]:
kmean_cpu_start = time.perf_counter()
kmean_cpu_result = kmean_cpu(np_data, centroid)
kmean_cpu_end = time.perf_counter()

In [41]:
kmean_cpu_end - kmean_cpu_start

82.27566378999995

### RUN GPU

In [42]:
kmean_gpu_start = time.perf_counter()
kmean_cpu_result = kmean_gpu(np_data, centroid)
kmean_gpu_end = time.perf_counter()

In [43]:
kmean_gpu_end - kmean_gpu_start

0.6374512180000238